In [ ]:
import sys
sys.path.append('../')
from joblib import Parallel, delayed
import numpy as np
import code.image_conversions as imc



import jpegio as jio
from os import listdir, path,mkdir,makedirs
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import numpy.random as npr

import rawpy

from code.image_tools  import edge_crop,center_crop

from PIL import Image

# Develop image

In [ ]:
comp_info = [{'ac_tbl_no': 0,
  'component_id': 1,
  'dc_tbl_no': 0,
  'h_samp_factor': 1,
  'quant_tbl_no': 0,
  'v_samp_factor': 1}]

c_quant_95 = np.array([\
        [ 2,  1,  1,  2,  2,  4,  5,  6],\
        [ 1,  1,  1,  2,  3,  6,  6,  6],\
        [ 1,  1,  2,  2,  4,  6,  7,  6],\
        [ 1,  2,  2,  3,  5,  9,  8,  6],\
        [ 2,  2,  4,  6,  7, 11, 10,  8],\
        [ 2,  4,  6,  6,  8, 10, 11,  9],\
        [ 5,  6,  8,  9, 10, 12, 12, 10],\
        [ 7,  9, 10, 10, 11, 10, 10, 10]])

# Quant table at 85% (convert)
c_quant_85 = np.array([\
     [ 5,  3,  3,  5,  7, 12, 15, 18],\
     [ 4,  4,  4,  6,  8, 17, 18, 17],\
     [ 4,  4,  5,  7, 12, 17, 21, 17],\
     [ 4,  5,  7,  9, 15, 26, 24, 19],\
     [ 5,  7, 11, 17, 20, 33, 31, 23],\
     [ 7, 11, 17, 19, 24, 31, 34, 28],\
     [15, 19, 23, 26, 31, 36, 36, 30],\
     [22, 28, 29, 29, 34, 30, 31, 30]])

# Quant table at 75% (convert)
c_quant_75 = np.array([\
        [ 8,  6,  5,  8, 12, 20, 26, 31],\
        [ 6,  6,  7, 10, 13, 29, 30, 28],\
        [ 7,  7,  8, 12, 20, 29, 35, 28],\
        [ 7,  9, 11, 15, 26, 44, 40, 31],\
        [ 9, 11, 19, 28, 34, 55, 52, 39],\
        [12, 18, 28, 32, 41, 52, 57, 46],\
        [25, 32, 39, 44, 52, 61, 60, 51],\
        [36, 46, 48, 49, 56, 50, 52, 50]])

def convert_npy_to_jpeg(im_dct, outpath, imname, QF=100):
    im_dct = np.round(im_dct)


    #im_tmp = (Image.fromarray(im_dct*0+127)).convert('L')
    #im_tmp.save(path.join(outpath, imname))
    if QF==100:
        I_struct = jio.read(path.join('data/skeleton_QF100.jpg'))
    elif QF==95:
        I_struct = jio.read(path.join('data/jpeg_skeleton_QF95.jpg'))
    elif QF==85:
        I_struct = jio.read(path.join('data/jpeg_skeleton_QF85.jpg'))
    elif QF==75:
        I_struct = jio.read(path.join('data/jpeg_skeleton_QF75.jpg'))
    else:
        print("No available skeleton for QF {}".format(QF))
        return(None)
    
    I_struct.coef_arrays[0][:,:] = im_dct[:,:]
    I_struct.optimize_coding = True
    try:
        I_struct.write(path.join(outpath, imname))
    except Exception as e:
        print("Failed to save jpeg")

In [ ]:
comp_info = [{'ac_tbl_no': 0,
  'component_id': 1,
  'dc_tbl_no': 0,
  'h_samp_factor': 1,
  'quant_tbl_no': 0,
  'v_samp_factor': 1}]

def develop_jpeg(grey, outpaths, imname, QFs=[100], save_precover=False):
    for i,QF in enumerate(QFs):
            if  QF == 100:
                c_quant = np.ones((8,8))
            elif QF == 95:
                c_quant = quant_95
            elif QF == 85:
                c_quant = quant_85
            elif QF == 75:
                c_quant = quant_75
            if save_precover:
                    im_dct = imc.compute_dct_domain(grey, c_quant)
                    (h,w)= im_dct.shape
                    im_dct = im_dct[0:8*(h//8), 0:8*(w//8)]
                    np.save(path.join(outpaths[i], 'precover', imname), im_dct)
                    im_dct = np.round(im_dct)
            else:
                    im_dct = imc.compute_jpeg_domain(grey, c_quant)
                    (h,w)= im_dct.shape
                    im_dct = im_dct[0:8*(h//8), 0:8*(w//8)]
                    im_dct = np.round(im_dct)


            convert_npy_to_jpeg(im_dct, outpaths[i], imname, QF=QF)


In [ ]:
def develop_image(i, params, idx_x, idx_y, RAW_path, jpeg_path, rf=3):


    if not path.isdir(path.join(jpeg_path, 'precover')):
        makedirs(path.join(jpeg_path, 'precover'))

    RAW_im = rawpy.imread(RAW_path)
    f = str(i)

    rgb = RAW_im.postprocess(params)
    (h,w) = RAW_im.raw_image_visible.shape

    crop_shape = 264

    grey = center_crop(imc.rgb2gray(rgb/(2**16-1)*255),1920, 1920)[idx_x:idx_x+crop_shape*rf, idx_y:idx_y+crop_shape*rf]

    if rf !=1:
        res_im = np.array(Image.fromarray(grey[:,:]).resize((crop_shape, crop_shape), Image.LANCZOS))
        develop_jpeg(res_im, [jpeg_path], f+'.jpg', QFs=[100], save_precover=True)
    else:
        develop_jpeg(grey, [jpeg_path], f+'.jpg', QFs=[100], save_precover=True)

In [ ]:
paramsLIN=rawpy.Params(rawpy.DemosaicAlgorithm.LINEAR, half_size=False, four_color_rgb=False, 
                    use_camera_wb=False, use_auto_wb=False,user_wb=(1,1,1,1), 
                    output_color=rawpy.ColorSpace.raw, output_bps=16, 
                    user_flip=None, user_black=0, user_sat=None, 
                    no_auto_bright=True, auto_bright_thr=None, 
                    adjust_maximum_thr=0.0, bright=1.0, 
                    highlight_mode=rawpy.HighlightMode.Clip,  gamma=(1,1),
                    exp_shift=None, exp_preserve_highlights=0.0, no_auto_scale=False,
                    chromatic_aberration=None, bad_pixels_path=None,median_filter_passes=0)
paramsBosslike=rawpy.Params(rawpy.DemosaicAlgorithm.PPG, half_size=False, four_color_rgb=False, 
                    use_camera_wb=True, use_auto_wb=False,user_wb=(1,1,1,1), 
                    output_color=rawpy.ColorSpace.raw, output_bps=16, 
                    user_flip=None, user_black=0, user_sat=None, 
                    no_auto_bright=True, auto_bright_thr=None, 
                    adjust_maximum_thr=0.0, bright=1.0, 
                    highlight_mode=rawpy.HighlightMode.Clip,  gamma=(1,1),
                    exp_shift=None, exp_preserve_highlights=0.0, no_auto_scale=False,
                    chromatic_aberration=None, bad_pixels_path=None,median_filter_passes=0)

In [ ]:
noise_params = np.load('data/BOSSBase/BOSS_noise_params_robust.npy', allow_pickle=True).item()
dataset = np.load('data/BOSSBase/BOSSBase_dataset.npy', allow_pickle=True).item()
orient_dict = np.load('data/BOSSBase/orient_dict_BossBase.npy', allow_pickle=True).item()
dataset_max_val = np.load('data/BOSSBase/Camera_BOSS_max_val.npy', allow_pickle=True).item()
im_idx = np.load('data/BOSSBase/BOSS_edge_crop_idx.npy', allow_pickle=True).item()


In [ ]:
base_path = 'datasets/BOSSBase/'
RAW_path = path.join( base_path, 'RAW')
RAW_files = listdir(RAW_path)

# Remove M9
RAW_files = [f for f in RAW_files if dataset[f][0] != 'M9 Digital Camera']
RAW_files_path = [RAW_path + file for file in RAW_files if path.isfile(RAW_path + file)]

jpeg_path = path.join(base_path, 'Bosslike/QF100/')


if not path.isdir(jpeg_path):
    makedirs(jpeg_path)


The next functions are used to develop the same image as in the paper, using the same crops without having to use the costly *edge_crop* function by directly giving the cropping indices.

In [ ]:
def computeBOSS(f):
    fn = f.split('.')[0]
    idx_x, idx_y = im_idx[fn]
    idx_x, idx_y = int(idx_x), int(idx_y)
    develop_image(fn, paramsBosslike, idx_x, idx_y, path.join(RAW_path, f), jpeg_path,rf=3 )
def computeLIN(f):
    fn = f.split('.')[0]
    idx_x, idx_y = im_idx[fn]
    idx_x, idx_y = int(idx_x), int(idx_y)
    develop_image(fn, paramsLIN, idx_x, idx_y, path.join(RAW_path, f), jpeg_path,rf=1  )

In [ ]:
computeLIN(f=RAW_files[0]) 
computeBOSS(f=RAW_files[0]) 

In [ ]:
Parallel(n_jobs=15)(delayed(computeBOSS)(f=f) for f in tqdm(RAW_files))